# Scrape a table from a page with python

# Question 1

In [1]:
import pandas as pd

### Scraping Table Data From Websites

In [2]:
URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
tables = pd.read_html(URL)
df=tables[0]

### The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [3]:
df.rename(columns={'Postal Code': 'PostalCode','Borough':'Borough','Neighbourhood':'Neighbourhood'}, inplace=True)

### Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [4]:
df = df[df.Borough != "Not assigned"]
df.reset_index(drop=True,inplace=True)

### The rows with the same postal code area will be combined into one row with the neighborhoods separated with a comma 

In [5]:
df.groupby(['PostalCode']).sum()

,Borough,Neighbourhood
PostalCode,,
M1B,Scarborough,"Malvern, Rouge"
M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
M1E,Scarborough,"Guildwood, Morningside, West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae
...,...,...
M9N,York,Weston
M9P,Etobicoke,Westmount
M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."


### If a cell has a borough but a Not assigned  neighborhood, then the neighborhood will be the same as the borough

In [6]:
df[df.Neighbourhood == "Not assigned"].Neighbourhood=df[df.Neighbourhood == "Not assigned"].Borough
df.head(12)

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


### Use the .shape method to print the number of rows of your dataframe

In [7]:
df.shape

(103, 3)

# Question 2

### Input geographical coordinate data

In [8]:
df_geo_data=pd.read_csv("https://cocl.us/Geospatial_data")
df_geo_data.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)
df_geo_data.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476



### Combine two df and df_geo_data with the same postal code


In [9]:
df=df.merge(df_geo_data, how='inner', on='PostalCode',suffixes=(False, False))
df.head(12)

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


# Question 3

In [10]:
from sklearn.cluster import KMeans 
import matplotlib.pyplot as plt # plotting library
# backend for rendering plots within the browser
%matplotlib inline 
! pip install folium==0.5.0
import folium # plotting library
print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


#### Use geopy library to get the latitude and longitude values of Toronto.

In [11]:
! pip install geopy
import geopy # plotting library
print('geopy installed')
print('Libraries imported.')
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

geopy installed
Libraries imported.


In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>ny_explorer</em>, as shown below.

In [12]:
address = 'Toronto'
geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


### Create map of Toronto using latitude and longitude values and add markers to map

In [13]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Extract data with only boroughs that contain the word Toronto

In [14]:
toronto_data = df[df['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
toronto_data.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


### Each string in the column of Neighbourhood is split by sep and returned as a DataFrame of dummy/indicator variables.

In [15]:
df_neighbourhood=df['Neighbourhood'].str.get_dummies(sep=',')
df_neighbourhood.head()

,Adelaide,Agincourt North,Albion Gardens,Bathurst Quay,Beaumond Heights,Bloordale Gardens,Broadview North (Old East York),Cabbagetown,Chinatown,Clairlea,...,Victoria Village,West Deane Park,Westmount,Weston,Wexford,Willowdale,Woburn,Woodbine Heights,York Mills,York Mills West
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Calculate the frequency of each category of neighbourhood 

In [16]:
df_neig=pd.DataFrame(df_neighbourhood)
length=len(df_neighbourhood.sum())
total=df_neighbourhood.sum()
for i in range(length):
    df_neig.iloc[:,i].div(total[i])
df_neig

,Adelaide,Agincourt North,Albion Gardens,Bathurst Quay,Beaumond Heights,Bloordale Gardens,Broadview North (Old East York),Cabbagetown,Chinatown,Clairlea,...,Victoria Village,West Deane Park,Westmount,Weston,Wexford,Willowdale,Woburn,Woodbine Heights,York Mills,York Mills West
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
99,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
100,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
101,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### K-means clustering according to the neighbourhood

In [17]:
# set number of clusters
kclusters = 5
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(df_neighbourhood)
# check cluster labels generated for each row in the dataframe
kmeans.labels_ 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 3,
       0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], dtype=int32)